In [ ]:
'''
Analyze the frequency response of customized soft actuators
Created on 2023.01.23 based on 'FrequencySweep.ipynb'
'''

# To be commented for still figure format
# %matplotlib notebook
# %matplotlib notebook

# Import packages
import time
from os import walk
import os.path as ospa
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import mlab
import pandas as pd
import scipy.io as scio
from scipy import signal
# import seaborn as sns
import IPython.display as ipd
from IPython.core.display import HTML

''' Figure format'''
plt.rc('font', size=16, family='Verdana') # 'Tahoma', 'DejaVu Sans', 'Verdana'"
plt.rc('axes', edgecolor='k', linewidth=0.75, labelcolor='k')
plt.rc('axes.spines', **{'bottom':True, 'left':True, 'right':True, 'top':True})
plt.rcParams['xtick.top'] = True
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['errorbar.capsize'] = 4

''' Define Color Here '''
pltBlue = (32/255,120/255,180/255)
pltRed = (180/255,32/255,32/255)
pltgreen = ()

''' Suppress warnings '''
import warnings
warnings.filterwarnings('ignore')

''' 
General Settings
'''
AudioFs = 44100 # (Hz) Must match the sampling frequency of the audio output

'''
General Functions
'''
def aPlot(figName='', figsize=(14, 6), is3D=False, dpi=72):
    ax = []
    
    fig1 = plt.figure(figsize=figsize, dpi=dpi)
    
    fig1.suptitle(figName, fontsize=16)
    if (is3D):
        ax = fig1.add_subplot(111, projection='3d')
    else:
        ax = fig1.add_subplot(111)
        
    return ax, fig1

def decodeData(fileName, decodeFormat, frontCode='', rearCode='', isString=False):
    segStr = re.findall(frontCode+decodeFormat+rearCode, fileName)
    if segStr:
        decoded = re.findall(decodeFormat, segStr[0])[0]      
        if isString:
            return decoded  
        return float(decoded)
    return None

def lowpassSmooth(datain, cutFreqRatio=0.05, order=8):
    b, a = signal.butter(order, 2 * cutFreqRatio, btype='low')
    dataout = signal.filtfilt(b, a, datain)
    return dataout

def movAvgSmooth(datain, winLen=100):
    dataout = np.convolve(datain, np.ones(winLen)/winLen, mode='same')
    return dataout

def onsetSegmentation(datain, segIntervalSamp, cutFreqRatio=0, disp=False, threshold=0.5):
    if(cutFreqRatio > 0):
        smoothSig = lowpassSmooth(abs(datain), cutFreqRatio=cutFreqRatio)
    
    smoothSig = smoothSig - smoothSig[0]
    
    samp = np.arange(len(datain))
    
#     maxValue = np.max(smoothSig)
    maxValue = np.sqrt(np.mean(np.square(smoothSig)))
    
    segPointInd = np.squeeze(np.argwhere(smoothSig > threshold * maxValue)) # Find value larger than 25% of peak as valid segment data point
    segGapInd = np.squeeze(np.argwhere(np.diff(segPointInd) > segIntervalSamp))# Index of point where gap is longer than the predetermined interval

    startInd = np.array(segPointInd[segGapInd+1])
    startInd = np.insert(startInd, 0, segPointInd[0])

    endInd = np.array(segPointInd[segGapInd])
    endInd = np.append(endInd, segPointInd[-1])
    
    if disp:
        ax0, _ = aPlot(); 
        ax0.plot(samp, datain, color='tab:grey'); 
        axb = ax0.twinx() 
        axb.plot(samp, smoothSig, color='tab:blue')
        ax0.plot(startInd, np.zeros(startInd.shape), '*r')
        ax0.plot(endInd, np.zeros(endInd.shape), '*c')
        plt.show();
        
    return zip(startInd, endInd)

def onsetSegmentation2(datain, segIntervalSamp, winLen=8000, disp=False, threshold=0.8):
    if(winLen > 0):
        smoothSig = movAvgSmooth(np.square(datain), winLen=winLen)
    
    smoothSig = smoothSig - smoothSig[0]
    
    samp = np.arange(len(datain))
    
    maxValue = np.sqrt(np.mean(np.square(smoothSig)))
    
    ax0 = None
    if disp:
        ax0, _ = aPlot(); 
        ax0.plot(samp, datain, color='tab:grey'); 
        axb = ax0.twinx() 
        axb.plot(samp, smoothSig, color='tab:blue')   
    
    segPointInd = np.squeeze(np.argwhere(smoothSig > threshold * maxValue)) # # Find value by thresholdiing as valid points
    segGapInd = np.squeeze(np.argwhere(np.diff(segPointInd) > segIntervalSamp))# Index of point where gap is longer than the predetermined interval

    startInd = np.array(segPointInd[segGapInd+1])
    startInd = np.insert(startInd, 0, segPointInd[0])
    
    if disp:
        axb.plot(segPointInd, maxValue*np.ones(segPointInd.shape), '.',color='tab:orange')
        ax0.plot(startInd, np.zeros(startInd.shape), '*r')
        plt.show();
        
    return startInd

def periodicSegmentation(datain, Fs, segmentNum, segmentTotalDuration, vibrationDuration, threshold=0.8):
    segIndices = onsetSegmentation2(datain, 7000, threshold=threshold)
    
    if segIndices is None:
        print("Segmentation failed")
        return None
    
    segStartInd = np.arange(0, segmentNum*segmentTotalDuration * Fs, segmentTotalDuration * Fs, dtype=int) + segIndices[0]

    return zip(segStartInd, segStartInd + int(vibrationDuration*Fs))
    

def sinSignal(sinFreq=250, sinDuration=6.0, Fs=48000, isUnipolar=False, completeCycle=True): # Generate sinusoid signals with percentage range and zero start
    # Sinwave Frequency (Hz), Time duration (sec), Sampling Frequency (Hz), is Unipolar Signal or Bipolar, ensure completed cycle by adjusting time duration
    
    oneCycleDuration = 1.0/sinFreq # (secs) Time duration of one cycle
    
    adjustedDuration = np.ceil(sinDuration/oneCycleDuration)*oneCycleDuration
    
    t = np.arange(int(adjustedDuration * Fs)) / Fs
    
    if (isUnipolar):
        y = -0.5 * np.cos(2 * np.pi * sinFreq * t) + 0.5 # Unipolar: y starts from 0 and increases to 1.0 amplitude
    else:
        y = np.sin(2 * np.pi * sinFreq * t) # Bipolar: y starts from 0 and increases to +1.0, then down to -1.0 amplitude
    
    y = np.insert(y, 0, 0.0)

    return y, t

def dataMean(df, selectFreq):
    freqNum = len(selectFreq)
    
    rmsEnergyMean = []
    peakFreqMagMean = []
    for i in range(freqNum):
        ind = (df['ActuateFreq'] == selectFreq[i])

        if len(ind) > 0:
            rmsEnergyMean.append(np.mean(np.asarray(df.loc[ind, 'RMSEnergy'].values[0], dtype='float64')))
            peakFreqMagMean.append(np.mean(np.asarray(df.loc[ind, 'PeakFreqMag'].values[0], dtype='float64')))
        else:
            rmsEnergyMean.append(None)
            peakFreqMagMean.append(None)
    return rmsEnergyMean, peakFreqMagMean

In [ ]:
'''
Analysis of Laser Vibrometer Measurement (Added on 2023.01.19)
'''
# DataPath = './SoftActuator202301'
# DataPath = './FreqResponseData20230127'
DataPath = './19042023'

# VibroGo data format
VibroGoHeadRowIndex = 3
TimeLabel = '[ s ]'
VibVelocityLabel = '[ m/s ]'

# Settings for data segmentation
# segmentLen = segmentTime + segmentInterval # (secs) 
# selectedStartT = 0.5 # (secs) Selected the start time of the window for analysis
# selectedLen = 2 # (secs) Selected the length of the window for analysis

''' Data preprocess and segmentation '''
FSdata = []

for root, directories, files in walk(DataPath):
    for fileName in files:
        
        actuatorName = decodeData(root, '\w+', frontCode='Actuator', rearCode='', isString=True)
        
        actuatorLength = decodeData(fileName, '[\d+\.]*\d+', frontCode='L', rearCode='')
        
        actuatorWidth = decodeData(fileName, '[\d+\.]*\d+', frontCode='W', rearCode='')
        
        infillVolume = decodeData(fileName, '[\d+\.]*\d+', frontCode='', rearCode='mL')
        
        actuateFreq = decodeData(fileName, '[\d+\.]*\d+', rearCode='Hz') 
        
        actuateVoltageLevel= decodeData(fileName, '\d+', frontCode='V')
        
        signaltype = re.search("sin", fileName)
        if signaltype is None:
            signaltype = re.search("pul", fileName)
        if (signaltype is not None):
            signaltype = signaltype[0]
        
        trialNum = decodeData(fileName, '\d+', frontCode='trial') 
        if(trialNum is None):
            trialNum = 0;

        print("%s, L%f, W%f, infill%f, freq%f, V%d, %s, trial%d" % (actuatorName, actuatorLength, actuatorWidth,
                                                                infillVolume, actuateFreq, actuateVoltageLevel, signaltype, 
                                                                    trialNum))
        
        metaData = pd.read_csv(ospa.join(root, fileName), header=None, nrows=VibroGoHeadRowIndex, sep='\t')
        display(HTML(metaData.to_html(index=False, header=None)))
        readData = pd.read_csv(ospa.join(root, fileName), header = VibroGoHeadRowIndex, sep='\t')
        
        t = readData[TimeLabel].to_numpy(dtype='float64')
        vibVelocity = readData[VibVelocityLabel].to_numpy(dtype='float64')
        
        Fs = 1/np.mean(np.diff(t))

        fig,ax = plt.subplots(2,1,dpi=72, figsize=(14,4))
        ax[0].plot(t, vibVelocity, color='tab:grey')
        
        
        ''' Data Segmentation '''
#         segIndPair = onsetSegmentation(vibVelocity, 7000, cutFreqRatio=0.004)

        segIndPair = periodicSegmentation(vibVelocity, Fs, 6, 4, 2.5, threshold=0.8)
        
        rmsEnergy = []
        freqMagValue = []
        measuredPeakFreq = []
        for i0,i1 in segIndPair:
            ax[0].plot(t[i0:i1], vibVelocity[i0:i1], '-', color='tab:orange')
            
            rmsEnergy.append(np.sqrt(np.mean(np.square(vibVelocity[i0:i1])))) # Compute RMS energy
            
            spectr, f, _ = ax[1].magnitude_spectrum(vibVelocity[i0:i1], Fs=Fs, color='tab:grey', window=mlab.window_none)
            
            f_desired = (abs(f - actuateFreq)).argmin() # Get the frequency index around the desired frequency freqMagValue.append(spectr[f_i]) 
            
            ind_left = max(f_desired-50,0)
            ind_right = (f_desired+50)
            spectrSegment = spectr[ind_left:ind_right]

            idx = spectrSegment.argmax() + ind_left

            freqMagValue.append(np.mean(spectr[idx-2:idx+2])) # Compute the spectral magnitude of desired frequency #  
    
            measuredPeakFreq.append(f[idx]) # Actual peak frequency near the desired frequency
            
            ax[1].plot(f[idx], spectr[idx], '.r') 
            
        ax[1].set_xlim([0, 802]) # (Hz) Zoom spectrum plot at selected frequency range
        
        fig.suptitle("%s [%s] - Desired freq = %.2f Hz - Measured peak freq = %.2f Hz" % 
                     (actuatorName, signaltype, actuateFreq, np.mean(measuredPeakFreq)))

        FSdata.append([actuatorName, actuatorLength, actuatorWidth, infillVolume, signaltype, actuateFreq, actuateVoltageLevel, trialNum,
                       np.mean(measuredPeakFreq), np.array(rmsEnergy), np.array(freqMagValue)])
        
print("Sampling frequency = %.2fHz" % Fs)    
        
FSdata = pd.DataFrame(FSdata, columns = ['ActuatorName','ActuatorLength','ActuatorWidth','InfillVolume','SignalType','ActuateFreq',
                                         'ActuateVoltageLevel','TrialNum','MeasuredPeakFreq','RMSEnergy','PeakFreqMag'])
display(FSdata)

In [ ]:
''' Visulize Frequency Response (pulse vs sin) '''
selectFreq = [1, 4, 10, 20, 40, 80, 160, 240, 400]

''' Compare spectrum '''
ax0,fig0 = aPlot(figName='Actuator Frequency Response (RMS Energy)')

rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[(FSdata['ActuatorName'] == 'Actuator9L20W15') & 
                                                     (FSdata['SignalType'] == 'pul')], selectFreq)
ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator9 (0.06ml) pulse')

rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[(FSdata['ActuatorName'] == 'Actuator9L20W15') & 
                                                     (FSdata['SignalType'] == 'sin')], selectFreq)
ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator9 (0.06ml) sin')

ax0.legend()

In [ ]:
''' Visulize Frequency Response'''
selectFreq = [4, 10, 20, 40, 80, 160, 240, 400, 800]

''' Compare spectrum '''
ax0,fig0 = aPlot(figName='Actuator Frequency Response (RMS Energy)')
# ax1,fig1 = aPlot(figName='Actuator Frequency Response (Peak Freq Magnitude)')

rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator1L20W15'], selectFreq)
ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator1(0.1ml)')
# ax1.plot(selectFreq, peakFreqMagMean, '.-', label='Actuator1(0.1ml)')

rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator2L20W15'], selectFreq)
ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator2(0.1ml)')
# ax1.plot(selectFreq, peakFreqMagMean, '.-', label='Actuator2(0.1ml)')

rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator3L20W15'], selectFreq)
ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator3(0.1ml)')
# ax1.plot(selectFreq, peakFreqMagMean, '.-', label='Actuator2')
rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator4L20W15'], selectFreq)
ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator4(0.2ml)')

rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator5L20W15'], selectFreq)
ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator5(0.2ml)')

rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator6L20W15'], selectFreq)
ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator6(0.2ml)')

rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator7L20W15'], selectFreq)
ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator7(0.1ml)')

ax0.legend()

# plt.legend(['Actuator1(0.1ml)','Actuator2(0.1ml)', 'Actuator3(0.1ml)', 'Actuator4(0.2ml)', 'Actuator5(0.2ml)', 'Actuator6(0.2ml)'])

In [ ]:
print(peakFreqMagMean)

In [ ]:
# ''' Visulize Frequency Response'''


# selectFreq = [2, 4, 10, 20, 40, 60, 80, 100, 120]


# ''' Compare spectrum '''
# ax0,fig0 = aPlot(figName='Actuator Frequency Response (RMS Energy)')
# # fig0.set_figuresize((14,3))
# ax1,fig1 = aPlot(figName='Actuator Frequency Response (Peak Freq Magnitude)')

# rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator1L20W15'], selectFreq)
# ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator1')
# ax1.plot(selectFreq, peakFreqMagMean, '.-', label='Actuator1')
# plt.legend(['Actuator1','Actuator2'])

# rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator2L20W15'], selectFreq)
# ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator2')
# ax1.plot(selectFreq, peakFreqMagMean, '.-', label='Actuator2')

# rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator3L20W15'], selectFreq)
# ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator3')
# ax1.plot(selectFreq, peakFreqMagMean, '.-', label='Actuator3')

# # rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator4L20W20'], selectFreq)
# # ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator4')
# # ax1.plot(selectFreq, peakFreqMagMean, '.-', label='Actuator4')

# # rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator5L20W15'], selectFreq)
# # ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator5')
# # ax1.plot(selectFreq, peakFreqMagMean, '.-', label='Actuator5')

# # rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator6L20W15'], selectFreq)
# # ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator6')
# # ax1.plot(selectFreq, peakFreqMagMean, '.-', label='Actuator6')

# # rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator7L20W15'], selectFreq)
# # ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator7')
# # ax1.plot(selectFreq, peakFreqMagMean, '.-', label='Actuator7')

# # rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator14L20W15'], selectFreq)
# # ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator14')
# # ax1.plot(selectFreq, peakFreqMagMean, '.-', label='Actuator14')

# # rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator9L20W15'], selectFreq)
# # ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator9')
# # ax1.plot(selectFreq, peakFreqMagMean, '.-', label='Actuator9')

# # rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator10L20W15'], selectFreq)
# # ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator10')
# # ax1.plot(selectFreq, peakFreqMagMean, '.-', label='Actuator10')

# # rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator11L20W15'], selectFreq)
# # ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator11')
# # ax1.plot(selectFreq, peakFreqMagMean, '.-', label='Actuator11')

# rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator12L20W15'], selectFreq)
# ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator12')
# ax1.plot(selectFreq, peakFreqMagMean, '.-', label='Actuator12')

# rmsEnergyMean, peakFreqMagMean = dataMean(FSdata.loc[FSdata['ActuatorName'] == 'Actuator13L20W15'], selectFreq)
# ax0.plot(selectFreq, rmsEnergyMean, '.-', label='Actuator13')
# ax1.plot(selectFreq, peakFreqMagMean, '.-', label='Actuator13')

# ax0.legend();
# ax1.legend();

# ax0.set_xlabel('Frequency (Hz)')
# ax0.set_ylabel('Amplitude (m/s)')
# ax1.set_xlabel('Frequency (Hz)')
# ax1.set_ylabel('Amplitude (m/s)')

# # ax.set_ylim([-40, 0]);